<a href="https://colab.research.google.com/github/itsbimaps14/SentimentAnalysisforCampaignProduct/blob/main/5b_LoadModelandRunFile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, sys
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/MyDrive/DATA_UM/ultramilk/label/data_generate/

Mounted at /content/drive/
/content/drive/MyDrive/DATA_UM/ultramilk/label/data_generate


In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 84.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.5 MB/s eta 0:00:00


In [ ]:
# Import necessary libraries
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch

import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap

from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

# Preprocessing Data
import re
import string

# Dataframe
import csv

%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

rcParams['figure.figsize'] = 12, 8

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [ ]:
from transformers import AutoTokenizer, AutoModel

MODEL_NAME = 'indolem/indobert-base-uncased'

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
bert_model = AutoModel.from_pretrained(MODEL_NAME)

In [ ]:
# Build the Sentiment Classifier class
class SentimentClassifier(nn.Module):

    # Constructor class
    def __init__(self, n_classes):
        super(SentimentClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(MODEL_NAME)
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Linear(self.bert.config.hidden_size, n_classes)

    # Forward propagaion class
    def forward(self, input_ids, attention_mask):
        _, pooled_output = self.bert(
          input_ids=input_ids,
          attention_mask=attention_mask,
          return_dict=False
        )
        #  Add a dropout layer
        output = self.drop(pooled_output)
        return self.out(output)

In [ ]:
class_names = ['negative', 'neutral', 'positive']
MAX_LEN = 75

In [ ]:
model = SentimentClassifier(len(class_names))
# IF Load with GPU
#model.load_state_dict(torch.load('best_model_state.bin'))
# IF Load with CPU
model.load_state_dict(torch.load('best_model_state.bin', map_location=torch.device('cpu')))
model = model.to(device)

In [ ]:
!pip install Sastrawi
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

# Indonesian Stopword by Sastrawi
factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()

# import StemmerFactory class
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 3.1 MB/s eta 0:00:00


In [ ]:
def clean_text_predict(text) :
    # REMOVE PUNCTUATION
    ## Remove hashtag dan @
    ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",text).split())
    ## Remove punctiation
    text  = "".join([char for char in text if char not in string.punctuation])
    ## Remove angka
    text = re.sub('[0-9]+', '', text)
    ## Remove website link
    text = re.sub(r'http\S+', '', text)

    # REMOVE STOPWORD
    text = stopword.remove(text)

    # TEXT STEMMING
    text = stemmer.stem(text)

    return text

In [ ]:
review_text = "Wah sangat disayangkan kenapa ba nya stray kids kyk ga ada artist indonesia lainnya."
txt_clean = clean_text_predict(review_text)

encoded_review = tokenizer.encode_plus(
    txt_clean,
    max_length=MAX_LEN,
    add_special_tokens=True,
    return_token_type_ids=False,
    pad_to_max_length=True,
    return_attention_mask=True,
    return_tensors='pt',
)
input_ids = encoded_review['input_ids'].to(device)
attention_mask = encoded_review['attention_mask'].to(device)

output = model(input_ids, attention_mask)
_, prediction = torch.max(output, dim=1)

print(f'Review text\t: {review_text}')
print(f'Clean text\t: {txt_clean}')
print(f'Sentiment\t: {class_names[prediction]}')

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Review text	: Wah sangat disayangkan kenapa ba nya stray kids kyk ga ada artist indonesia lainnya.
Clean text	: wah sangat sayang ba nya stray kids kyk ga artist indonesia lain
Sentiment	: negative


In [ ]:
def predict_sentiment(text) :
  # Preprocess Data Text
  txt_clean = clean_text_predict(text)

  # Set Parameter Input
  encoded_review = tokenizer.encode_plus(
      txt_clean,
      max_length=MAX_LEN,
      add_special_tokens=True,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt',
  )

  # Transform Data to Tensors
  input_ids = encoded_review['input_ids'].to(device)
  attention_mask = encoded_review['attention_mask'].to(device)

  # Predict
  output = model(input_ids, attention_mask)
  _, prediction = torch.max(output, dim=1)

  # Return Data
  return class_names[prediction]

In [ ]:
df = pd.read_excel('data_labelled_dummy.xlsx', index_col=0)
df.head(5)

,date,is-retweet,name,username,profile_id,comments,retweets,quotes,likes,text,text_stemming,text_token,polarity
0,"Oct 31, 2023 · 1:04 PM UTC",True,Make Your Move!,@ultramym,1197372091978276864,6,783,57,1809,"Btw, bakal ada premiere video #UltraMilkXStray...",btw bakal premiere video ultramilkxstraykids b...,"['btw', 'bakal', 'premiere', 'video', 'ultrami...",1
1,"Nov 2, 2023 · 11:45 PM UTC",False,Lau^^,@Seungminlock,1645021719734923264,0,0,0,1,"Naik krl di pagi hari yg cerah, sambil ngeliat...",naik krl pagi hari yg cerah ngeliatin barudak ...,"['naik', 'krl', 'pagi', 'hari', 'yg', 'cerah',...",3
2,"Nov 1, 2023 · 1:31 AM UTC",True,FELIX INDONESIA🇲🇨,@LeeFelixID,1641970160700657670,3,839,119,2288,[🐣] Wah pagi-pagi udah disambut @Stray_Kids d...,wah pagipagi udah sambut straykids sarinah ran...,"['wah', 'pagipagi', 'udah', 'sambut', 'strayki...",3
3,"Nov 2, 2023 · 10:20 AM UTC",True,Miho₋stay🐥🇯🇵🇮🇩,@Miho825_jkt,1665330990305406976,7,4,0,76,本日のウルトラミルクチャレンジは、 RASA KARAMEL=キャラメル味！ とっても甘く...,rasa karamel im japanese living in indonesia i...,"['rasa', 'karamel', 'im', 'japanese', 'living'...",2
4,"Nov 2, 2023 · 5:12 PM UTC",True,cilul 樂 ★,@youtiflul,1716136049519525888,0,2,0,4,oh that's why they're holding the mocha flavor...,thats why theyre holding the mocha flavored on...,"['thats', 'why', 'theyre', 'holding', 'the', '...",1


In [ ]:
df['prediction'] = df['text_stemming'].apply(lambda x: predict_sentiment(x))
df.head

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


<bound method NDFrame.head of                             date  is-retweet               name  \
0     Oct 31, 2023 · 1:04 PM UTC        True    Make Your Move!   
1     Nov 2, 2023 · 11:45 PM UTC       False              Lau^^   
2      Nov 1, 2023 · 1:31 AM UTC        True  FELIX INDONESIA🇲🇨   
3     Nov 2, 2023 · 10:20 AM UTC        True     Miho₋stay🐥🇯🇵🇮🇩   
4      Nov 2, 2023 · 5:12 PM UTC        True          cilul 樂 ★   
...                          ...         ...                ...   
1265                Nov 14, 2023       False         Dummy-Data   
1266                Nov 14, 2023       False         Dummy-Data   
1267                Nov 14, 2023       False         Dummy-Data   
1268                Nov 14, 2023       False         Dummy-Data   
1269                Nov 14, 2023       False         Dummy-Data   

           username           profile_id  comments  retweets  quotes  likes  \
0         @ultramym  1197372091978276864         6       783      57   1809   
1     @

In [ ]:
df.to_excel("data_labelled_dummy_predicted.xlsx")